# 第９章RNN, CNN

### 80.ID番号への変換  
問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に1，2番目に頻出する単語に2，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて0とせよ．

In [1]:
import pandas as pd

train = pd.read_csv('../ss6/csv/train.csv', sep='\t')
valid = pd.read_csv('../ss6/csv/valid.csv', sep='\t')
test = pd.read_csv('../ss6/csv/test.csv', sep='\t')
df = pd.concat([train, valid, test], axis=0)
df.head()

,TITLE,PUBLISHER,CATEGORY
0,L'wren Scott - L'Wren Scott's housekeeper reve...,Contactmusic.com,e
1,Australia Set to Fund Second Clean-Energy Proj...,Businessweek,b
2,Google Brings Android to Smartwatches in Mobil...,Businessweek,t
3,Tina Fey - Tina Fey Dispels Mean Girls Sequel ...,Contactmusic.com,e
4,Rick Rubin - Rick Rubin Takes An Ice Bath For ...,Contactmusic.com,e


In [2]:
df.shape

(13340, 3)

In [3]:
display(train.head())
print(train.shape)

,TITLE,PUBLISHER,CATEGORY
0,L'wren Scott - L'Wren Scott's housekeeper reve...,Contactmusic.com,e
1,Australia Set to Fund Second Clean-Energy Proj...,Businessweek,b
2,Google Brings Android to Smartwatches in Mobil...,Businessweek,t
3,Tina Fey - Tina Fey Dispels Mean Girls Sequel ...,Contactmusic.com,e
4,Rick Rubin - Rick Rubin Takes An Ice Bath For ...,Contactmusic.com,e


(10672, 3)


In [8]:
import string
from collections import defaultdict

words_count = defaultdict(int)
table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
for text in train['TITLE']:
    for word in text.translate(table).split():
        words_count[word] += 1
words_count = sorted(words_count.items(), key=lambda x: x[1], reverse=True)
w2id = {w: i+1 for i, (w, c) in enumerate(words_count) if c>= 2}

def text2id(text):
    table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    words_list = text.translate(table).split()
    return [w2id.get(word, 0) for word in words_list]

train['TITLE_ID'] = train['TITLE'].apply(lambda x: text2id(x))
train.head()

,TITLE,PUBLISHER,CATEGORY,TITLE_ID
0,L'wren Scott - L'Wren Scott's housekeeper reve...,Contactmusic.com,e,"[214, 988, 258, 214, 559, 258, 2, 5171, 172, 1..."
1,Australia Set to Fund Second Clean-Energy Proj...,Businessweek,b,"[506, 277, 1, 1338, 360, 3512, 790, 1914, 18, ..."
2,Google Brings Android to Smartwatches in Mobil...,Businessweek,t,"[66, 2651, 1711, 1, 0, 3, 292, 837]"
3,Tina Fey - Tina Fey Dispels Mean Girls Sequel ...,Contactmusic.com,e,"[3043, 3044, 3043, 3044, 0, 2332, 711, 883, 2102]"
4,Rick Rubin - Rick Rubin Takes An Ice Bath For ...,Contactmusic.com,e,"[1339, 6690, 1339, 6690, 637, 480, 791, 0, 23,..."


### 81.RNNによる予測. 
ID番号で表現された単語列$\textbf{x}=(x_1, x_2, \dots, x_T)$がある．ただし，Tは単語列の長さ，$x_t\in\mathbb{R}^V$は単語のID番号のone-hot-vecotrである（Vは単語の総数である）．RNNを用い，単語列$\textbf{x}$からカテゴリyを予測するモデルとして，時式を実装せよ．  
$$ \overrightarrow{h_0} = 0, \\ \overrightarrow{h_t} = \overrightarrow{RNN}(emb(x_t), \overrightarrow{h_{t_1}}),\\ y = softmax(W^{(yh)}\overrightarrow{h_T}+b^{(y)})$$
ただし，$emb(x)\in\mathbb{R}^{d_w}$は単語の埋め込み，$\overrightarrow{h_t}\in\mathbb{R}^{d_n}$は時刻tの隠れ常態ベクトル，$\overrightarrow{RNN}(x, h)$は入力xと前時刻の隠れ状態ベクトルからカテゴリを予測するための行列，$b^{(y)}\in\mathbb{R}^L$はバイアス項である．RNNユニット$\overrightarrow{RNN}(x, h)$二はさまざまな構成が考えられるが，典型例として字式が挙げられる．  
$$ \overrightarrow{RNN}(x, h) = g(W^{(hx)}x + W^{(hh)}h + b^{(hh)})$$  
ただしgはtanhやReLUなどの活性化関数である．  
なお，この問題ではパラメータの学習を行わず，ランダムに初期化されたパラメータでyを計算するだけで良い．次元数などのハイパーパラメータは,$d_w = 300, d_h=50$など，適当な値に設定せよ．

In [ ]:
import torch
from torch import nn
from torch.nn import fanctional as F


### 82.SGDによる学習  
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，問題81で構築したモデルを学習せよ．訓練データ上の損失と正解率，評価データ上の損失と正解率を表示しながらモデルを学習し，適当な基準（例えば10エポックなど）で終了させよ

### 83.ミニバッチ化・GPU上での学習  
問題82のコードを改変し，B
事例ごとに損失・勾配を計算して学習を行えるようにせよ（B
の値は適当に選べ）．また，GPU上で学習を実行せよ．

### 84.単語ベクトルの導入  
事前学習済みの単語ベクトル（例えば，Google Newsデータセット（約1,000億単語）での学習済み単語ベクトル）で単語埋め込みemb(x)
を初期化し，学習せよ．

### 85.双方向RNN・多層化  
順方向と逆方向のRNNの両方を用いて，入力テキストをエンコードし，モデルを学習せよ．  
$$ \overleftarrow{h_{T+1}} = 0,\\ \overleftarrow{h_t}=\overleftarrow{RNN}(emb(x_t), \overleftarrow{h_{t+1}}),\\ y = softmax(W^{(yh)}[\overrightarrow{h_T};\overleftarrow{h_1}]+b^{(y)})$$

ただし，[a;b]はベクトルaとbの連結を表す．さらに，双方向RNNを多層化して実験せよ．

### 86.CNN  
ID番号で表現された単語列$\textbf{x} = (x_1, x_2, \dots, x_T)$がある．ただし，Tは単語列の長さ，$x_t\in\mathbb{R}^V$は単語のID番号のone-hot-vectorである． CNNを用いて，単語列$\textbf{x}$からカテゴリyを予測するモデルを実装せよ．  
ただし，CNNのこうせいは以下の通りとする．  
・単語埋め込みの次元数: $d_w$  
・畳み込みのフィルターのサイズ:3トークン  
・畳み込みのストライド:1トークン  
・畳み込みのパディング:あり  
・畳み込みの演算後の各時刻のベクトルの次元数:$d_n$  
・畳み込み演算後に最大値プーリングを適用し，入力文を$d_n$次元の隠れベクトルで表現．  
すなわち，時刻tの特徴ベクトル$p_t\in\mathbb{R}^{d_n}$は次式で表される．  
$$p_t=g(W^{(px)}[emb(x_{t_1});emb(x_t);emb(x_{t+1})]+b^{(p)})$$
ただし，gはtanhやReLUなどの活性化関数で[a;b;c]はベクトルa,b,cの連結である．  
最大値プーリングでは，特徴ベクトルの次元ごとに全時刻における最大値を取り，入力文書の特徴ベクトル$c\in\mathbb{R}^{d_n}$を求める．c[i]でベクトルcのi番目の次元の値を表すことにすると，最大値プーリングは次式で表される．  
$$ c[i] = max_{1<=t<=T} \space p_t[i]$$
最後に，入力文書の特徴ベクトルcに線形変換とソフトマックス関数を適用し，カテゴリyを予測する．  
$$ y = softmax(W^{(yc)}c+b^{(y)})$$
なお，この問題ではモデルの学習を行わず，ランダムに初期化された重み行列でyを計算するだけで良い．

### 87. 確率的勾配降下法によるCNNの学習
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，問題86で構築したモデルを学習せよ．訓練データ上の損失と正解率，評価データ上の損失と正解率を表示しながらモデルを学習し，適当な基準（例えば10エポックなど）で終了させよ

### 88. パラメータチューニング
問題85や問題87のコードを改変し，ニューラルネットワークの形状やハイパーパラメータを調整しながら，高性能なカテゴリ分類器を構築せよ．

### 89. 事前学習済み言語モデルからの転移学習
事前学習済み言語モデル（例えばBERTなど）を出発点として，ニュース記事見出しをカテゴリに分類するモデルを構築せよ．